In [ ]:
!pip install unidecode

     |████████████████████████████████| 245kB 10.1MB/s 


In [ ]:
import sys 
import os
import glob
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
#dataset retrieved from https://github.com/katakonst/sentiment-analysis-tensorflow/datasets
PATH = "/content/drive/MyDrive/fb_comments/resources_for_sentiment/romanian_movie_reviews.csv"

In [ ]:
def read_reviews(path):
  data = pd.read_csv(path)
  data = data[['text', 'label']]
  data.label = data.label.apply(lambda x: 1 if x=="pos" else 0)
  print(data.label.value_counts())
  return data

In [ ]:
def preprocess_balancing(data): 
  #50/50 distribution
  data.text = data.text.apply(lambda x: unidecode(x))
  data.text = data.text.apply(lambda x: x.replace("\n", " "))
  positive_texts = data[data.label==1]
  negative_texts = data[data.label==0]
  data_balanced = pd.concat([positive_texts.sample(negative_texts.shape[0]), negative_texts], axis = 0)
  data_balanced = data_balanced.sample(frac = 1, random_state = 42)
  print(data_balanced.label.value_counts())
  return data_balanced
  

In [ ]:
data = read_reviews(PATH)
data = preprocess_balancing(data)

1    15653
0    11654
Name: label, dtype: int64
1    11654
0    11654
Name: label, dtype: int64


In [ ]:
print(type(data))
print(len(data))
data.head()

<class 'pandas.core.frame.DataFrame'>
23308


,text,label
5907,foarte multumit super,1
22651,suspendarea enorma a necredintei este necesara...,0
4265,aceasta serie de televiziune cu animatie este ...,1
25296,recenta editie dvd a good humor man eticheteaz...,0
22166,ce tot spui de violenta? daca mie mi-a placut ...,0


In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.text)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size: {}".format(vocab_size))
max_length = max([len(s.split()) for s in data.text])
print("Maximum token length: {}".format(max_length))

Vocabulary size: 58994
Maximum token length: 373


In [ ]:
def preprocess(X_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.lower(X_batch)
  X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-z]", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch


In [ ]:
data

,text,label
5907,foarte multumit super,1
22651,suspendarea enorma a necredintei este necesara...,0
4265,aceasta serie de televiziune cu animatie este ...,1
25296,recenta editie dvd a good humor man eticheteaz...,0
22166,ce tot spui de violenta? daca mie mi-a placut ...,0
...,...,...
15963,acest lucru ar trebui sa fie unul dintre cele ...,0
25574,"ca un adevarat canadian, evit mereu filmele ca...",0
13190,care a fost distrusa rapid de calitatea proast...,1
11617,o versiune actualizata a unei teme care a fost...,1


In [ ]:
x_all = tf.data.Dataset.from_tensor_slices(data.text)
y_all = tf.data.Dataset.from_tensor_slices(data.label)

In [ ]:
from collections import Counter
vocabulary = Counter()

for x_batch in x_all.batch(32).map(preprocess):
  for review in x_batch:
    vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:10]

[(b'de', 44594),
 (b'si', 28578),
 (b'a', 25533),
 (b'este', 20946),
 (b'nu', 19697),
 (b'in', 19629),
 (b'sa', 18917),
 (b'un', 18905),
 (b'ca', 18514),
 (b'o', 17919)]

In [ ]:
vocab_limit = 20000
truncated_vocab = [word for word, count in vocabulary.most_common()[:vocab_limit]]

In [ ]:
words = tf.constant(truncated_vocab)
word_ids = tf.range(len(truncated_vocab), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch

In [ ]:
x_all_preprocessed = x_all.batch(32).map(preprocess)
